In [1]:
import pandas as pd
df = pd.read_json("data/COCO/coco-easier.txt", 
                  lines=True)
df = df.sample(frac=1).reset_index(drop=True)
df[:5]



,all_categories,captions,category,file_name,img_id
0,"[cat, cat, bed]",[An orange cat laying on a white and brown cat...,cat,000000551815.jpg,551815
1,[toilet],[A toilet with an illuminated seat in a restro...,toilet,000000423971.jpg,423971
2,"[dining table, person, wine glass, wine glass,...",[A pizza sitting on top of a round cutting boa...,pizza,000000513041.jpg,513041
3,"[boat, clock, boat, boat]",[ a castle and the big ben clocktower next to ...,clock,000000342006.jpg,342006
4,"[cat, bottle]","[A cat climbing into a cat door on a wall., A ...",cat,000000456292.jpg,456292


In [2]:
inception_df = pd.read_json("data/inception_output.txt", lines=True)
inception_df.set_index('image',inplace=True)
inception_df[:5]

,vector
image,
000000173830.jpg,"[0.27057743072509705, 0.11477771401405301, 0.0..."
000000570664.jpg,"[0.252127289772033, 0.434070438146591, 0.22712..."
000000304396.jpg,"[0.24179434776306102, 0.08653920888900701, 0.1..."
000000441491.jpg,"[0.11056154966354301, 0.08768206834793, 0.0857..."
000000289059.jpg,"[0.23210205137729603, 0.1578049659729, 0.27599..."


In [3]:
import numpy as np

def concatenate(vector1, vector2):
    a = np.array(vector1)
    b = np.array(vector2)
    return np.concatenate((a, b))

In [4]:
import doc2vec_wrapped
doc2vec_vectors_series = doc2vec_wrapped.vectorize_content()
df["concat_vectors"] = df.apply(lambda row: concatenate(doc2vec_vectors_series[row['file_name']],
                                                       inception_df['vector'][row['file_name']]), axis=1)
df[:5]

c:\users\misko\miniconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2018-05-26 19:17:14,243 : INFO : collecting all words and their counts
2018-05-26 19:17:14,247 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-05-26 19:17:14,291 : INFO : collected 3702 word types and 10 unique tags from a corpus of 1503 examples and 65463 words
2018-05-26 19:17:14,293 : INFO : Loading a fresh vocabulary
2018-05-26 19:17:14,299 : INFO : min_count=5 retains 1058 unique words (28% of original 3702, drops 2644)
2018-05-26 19:17:14,300 : INFO : min_count=5 leaves 61217 word corpus (93% of original 65463, drops 4246)
2018-05-26 19:17:14,308 : INFO : deleting the raw counts dictionary of 3702 items
2018-05-26 19:17:14,318 : INFO : sample=0.001 downsamples 71 most-common words
2018-05-26 19:17:14,320 : INFO : downsampling leaves estima

,all_categories,captions,category,file_name,img_id,concat_vectors
0,"[cat, cat, bed]",[An orange cat laying on a white and brown cat...,cat,000000551815.jpg,551815,"[0.08489546179771423, 0.16070616245269775, 0.1..."
1,[toilet],[A toilet with an illuminated seat in a restro...,toilet,000000423971.jpg,423971,"[0.059617284685373306, 0.12213980406522751, 0...."
2,"[dining table, person, wine glass, wine glass,...",[A pizza sitting on top of a round cutting boa...,pizza,000000513041.jpg,513041,"[0.0127631900832057, 0.1059410348534584, 0.104..."
3,"[boat, clock, boat, boat]",[ a castle and the big ben clocktower next to ...,clock,000000342006.jpg,342006,"[0.05496961995959282, 0.014156408607959747, 0...."
4,"[cat, bottle]","[A cat climbing into a cat door on a wall., A ...",cat,000000456292.jpg,456292,"[0.17328797280788422, 0.10797213762998581, 0.1..."


In [5]:
from sklearn.neural_network import MLPClassifier
X = df["concat_vectors"]
y = df["category"]
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X, y)                         
MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False,
       epsilon=1e-08, hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)



ValueError: setting an array element with a sequence.

In [ ]:
clf.predict(df.loc[df['img_id'] == 125850].concat_vectors)